# Intelligent Travel Assistant - LangChain Agent Assignment

## 🎯 Assignment Objectives

In this assignment, you will build an **Intelligent Travel Assistant** that can:
1. **Search for local attractions** in any destination the user specifies
2. **Fetch current weather** for the destination
3. **Provide travel-ready information** combining both data sources

## 📋 What You'll Practice
- Creating custom tools with the `@tool` decorator
- Building a Tool-Calling Agent using `create_tool_calling_agent`
- Using `AgentExecutor` to run the agent
- Designing system prompts for specific use cases

## 🏗️ Architecture

```
User Query: "Tell me about Kolkata"
              │
              ▼
┌─────────────────────────────────────────┐
│         Travel Assistant Agent          │
├─────────────────────────────────────────┤
│  System Prompt: ReAct pattern for       │
│  travel queries + weather info          │
└─────────────────────────────────────────┘
              │
    ┌─────────┴─────────┐
    ▼                   ▼
┌───────────┐    ┌─────────────┐
│ Web Search │    │  Weather   │
│   Tool     │    │   Tool     │
└───────────┘    └─────────────┘
    │                   │
    ▼                   ▼
┌───────────┐    ┌─────────────┐
│ Tavily API│    │ WeatherAPI  │
└───────────┘    └─────────────┘
```

### **Complete Execution Workflow** :

1. **Input Reception** : User query received
2. **Agent Initialization** : ReAct agent starts reasoning loop
3. **Thought Process** : LLM analyzes what information is needed
4. **Tool Selection** : Chooses appropriate tools (search_web_extract_info, get_weather)
5. **Action Execution** : Calls selected tools with proper inputs
6. **Result Processing** : Processes tool outputs
7. **Iteration Decision** : Determines if more actions are needed
8. **Loop Continuation** : Repeats until sufficient information gathered
9. **Final Synthesis** : Combines all information into comprehensive answer
10. **Response Formatting** : Displays both full reasoning and final answer


---
## 🔧 Section 1: Environment Setup

In [ ]:
# =============================================================================
# INSTALL DEPENDENCIES (Run once, then comment out)
# =============================================================================
# Uncomment these lines if you need to install the packages:

# !pip install -qq langchain==0.3.14
# !pip install -qq langchain-openai==0.3.0
# !pip install -qq langchain-community==0.3.14
# !pip install -qq markitdown

In [ ]:
# =============================================================================
# LOAD ENVIRONMENT VARIABLES
# =============================================================================
# Your .env file should contain:
#   OPENAI_API_KEY=your_openai_key
#   TAVILY_API_KEY=your_tavily_key
#   WEATHER_API_KEY=your_weatherapi_key (from weatherapi.com)
# =============================================================================

import os
from dotenv import load_dotenv

load_dotenv()

# Verify that API keys are loaded
assert os.getenv('OPENAI_API_KEY'), "OPENAI_API_KEY not found in environment"
assert os.getenv('TAVILY_API_KEY'), "TAVILY_API_KEY not found in environment"

# Get Weather API key
WEATHER_API_KEY = os.getenv('WEATHER_API_KEY')

print("✅ Environment variables loaded successfully!")

---
## 🛠️ Section 2: Create Travel Tools

We need two tools for our travel assistant:

1. **`search_web_extract_info`**: Searches the web for local attractions using Tavily API
2. **`get_weather`**: Fetches current weather using WeatherAPI.com

### Key Concepts:
- The `@tool` decorator converts Python functions into LangChain tools
- **Docstrings are critical** - the LLM uses them to decide when to call each tool
- Tools should handle errors gracefully and return meaningful messages

In [ ]:
# =============================================================================
# TRAVEL ASSISTANT TOOLS
# =============================================================================
# Tool 1: Web Search - Finds local attractions using Tavily API
# Tool 2: Weather - Gets current weather using WeatherAPI.com
# =============================================================================

from langchain_core.tools import tool
from markitdown import MarkItDown
from langchain_community.tools.tavily_search import TavilySearchResults
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, TimeoutError
import requests
from warnings import filterwarnings

filterwarnings('ignore')

# -----------------------------------------------------------------------------
# TOOL 1: Web Search for Local Attractions
# -----------------------------------------------------------------------------
# Initialize Tavily search with basic settings (faster, good for location queries)
tavily_tool = TavilySearchResults(
    max_results=5,           # Return top 5 results
    search_depth='basic',    # Basic search is faster and sufficient for attractions
    include_answer=False,    # We'll process results ourselves
    include_raw_content=True # Include page content
)

# Configure HTTP session with browser-like headers to avoid bot detection
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br"
})

# MarkItDown converts web pages to readable text
md = MarkItDown(requests_session=session)

@tool
def search_web_extract_info(query: str) -> list:
    """
    Search the web for a query and extract useful information from the search links.
    
    Use this tool to find:
    - Local attractions and tourist spots
    - Things to do in a destination
    - Travel guides and recommendations
    
    Args:
        query (str): The search query (e.g., "local attractions in Kolkata")
        
    Returns:
        list: Extracted content from relevant web pages
    """
    print('🔍 Calling web search tool...')
    results = tavily_tool.invoke(query)
    docs = []

    def extract_content(url):
        """Helper function to extract content from a URL."""
        extracted_info = md.convert(url)
        text_title = extracted_info.title.strip()
        text_content = extracted_info.text_content.strip()
        return text_title + '\n' + text_content
    
    # Process URLs in parallel for faster extraction
    with ThreadPoolExecutor() as executor:
        for result in tqdm(results, desc="Extracting content"):
            try:
                future = executor.submit(extract_content, result['url'])
                content = future.result(timeout=15)  # 15-second timeout
                docs.append(content)
            except TimeoutError:
                print(f"⏰ Extraction timed out for url: {result['url']}")
            except Exception as e:
                print(f"❌ Error extracting from url: {result['url']} - {e}")

    return docs


# -----------------------------------------------------------------------------
# TOOL 2: Weather Information
# -----------------------------------------------------------------------------
# Uses WeatherAPI.com for current weather data

@tool
def get_weather(query: str) -> dict:
    """
    Get the current weather for a specified location.
    
    Use this tool whenever the user asks about:
    - Current weather conditions
    - Temperature at a destination
    - Weather for travel planning
    
    Args:
        query (str): The location/city name (e.g., "Kolkata", "Mumbai")
        
    Returns:
        dict: Weather data including temperature, condition, humidity, etc.
              Returns "Weather Data Not Found" if location is invalid
    """
    print('🌤️ Calling weather tool...')
    
    # WeatherAPI.com endpoint
    base_url = "http://api.weatherapi.com/v1/current.json"
    complete_url = f"{base_url}?key={WEATHER_API_KEY}&q={query}"

    response = requests.get(complete_url)
    data = response.json()
    
    # Check if location was found
    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"


print("✅ Travel tools created successfully!")

### Test the Weather Tool
Let's verify the weather tool works before building the agent.

In [ ]:
# =============================================================================
# TEST: Weather Tool
# =============================================================================
# Verify the tool works correctly before using it in the agent

result = get_weather.invoke("Kolkata")
print(f"Weather result type: {type(result)}")
print(f"Result: {result}")

---
## 🤖 Section 3: Build the Travel Assistant Agent

Now we'll create the agent with:
1. **System Prompt**: Defines the agent's behavior as a travel assistant
2. **Tool Binding**: Connect our search and weather tools to the LLM
3. **AgentExecutor**: Runtime that executes agent decisions

In [ ]:
# =============================================================================
# TEST: LLM Tool Calling Ability
# =============================================================================
# Before building the full agent, let's verify the LLM can correctly
# identify which tools to use for a travel query.
# =============================================================================

from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [search_web_extract_info, get_weather]

# Bind tools to the LLM
chatgpt_with_tools = chatgpt.bind_tools(tools)

# Test: This query should trigger BOTH tools (attractions + weather)
prompt = "Show me Local Attractions in Bangalore and The Weather in Bangalore"
response = chatgpt_with_tools.invoke(prompt)

print("Tool calls identified by the LLM:")
for tc in response.tool_calls:
    print(f"  • {tc['name']}: {tc['args']}")

In [ ]:
# =============================================================================
# CREATE THE TRAVEL ASSISTANT SYSTEM PROMPT
# =============================================================================
# The system prompt defines:
# 1. The agent's role as a travel assistant
# 2. The ReAct workflow (Thought → Action → Observation → Answer)
# 3. Special instruction to provide both attractions AND weather
# 4. Available tools and when to use them
# =============================================================================

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYS_PROMPT = """Act as a helpful travel assistant.
                You run in a loop of Thought, Action, PAUSE, Observation.
                At the end of the loop, you output an Answer.
                Use Thought to describe your thoughts about the question you have been asked.
                Use Action to run one of the actions available to you - then return PAUSE.
                Observation will be the result of running those actions.
                Repeat till you get to the answer for the given user query.
                
                IMPORTANT: When a user asks about a destination or location:
                1. Search for local attractions and tourist spots
                2. Get the current weather for that location
                3. Provide the attractions as bullet points with the weather information
                
                Use the following workflow format:
                  Question: the input task you must solve
                  Thought: you should always think about what to do
                  Action: the action to take which can be any of the following:
                            - break it into smaller steps if needed
                            - see if you can answer the given task with your trained knowledge
                            - call the most relevant tools at your disposal mentioned below in case you need more information
                  Action Input: the input to the action
                  Observation: the result of the action
                  ... (this Thought/Action/Action Input/Observation can repeat N times)
                  Thought: I now know the final answer
                  Final Answer: the final answer to the original input question

                
                Tools at your disposal to perform tasks as needed:
                  - get_weather: whenever user asks get the weather of a place.
                  - search_web_extract_info: whenever user asks for specific information or if you don't know the answer.
             """

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history", optional=True),
        ("human", "{query}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

print("✅ System prompt created!")

In [ ]:
# =============================================================================
# CREATE THE TRAVEL ASSISTANT AGENT
# =============================================================================
# Components:
# 1. LLM (gpt-4o-mini) - faster and more cost-effective for this use case
# 2. Tools - search_web_extract_info and get_weather
# 3. Prompt Template - our travel-focused system prompt
# 4. AgentExecutor - runs the agent and handles tool execution
# =============================================================================

from langchain.agents import create_tool_calling_agent, AgentExecutor

# Using gpt-4o-mini for faster, more cost-effective responses
chatgpt = ChatOpenAI(model="gpt-4o-mini", temperature=0)
tools = [search_web_extract_info, get_weather]

# Create the agent (decision-maker)
agent = create_tool_calling_agent(chatgpt, tools, prompt_template)

# Create the executor (runs the agent and tools)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    early_stopping_method='force',  # Force stop at max iterations
    max_iterations=10               # Prevent infinite loops
)

print("✅ Travel assistant agent created!")

---
## 🧪 Section 4: Test the Travel Assistant

Let's test our travel assistant with a destination query.

In [ ]:
# =============================================================================
# HELPER FUNCTION: Extract Final Answer
# =============================================================================
# The agent's response includes the full reasoning chain. This function
# extracts just the "Final Answer" portion for cleaner display.
# =============================================================================

import re

def extract_final_answer(agent_response):
    """
    Extract the Final Answer portion from the agent response.
    
    Args:
        agent_response (dict): The response from agent_executor.invoke()
    
    Returns:
        str: The final answer content only
    """
    output = agent_response['output']
    
    match = re.search(r'Final Answer:\s*(.*)', output, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return output  # Return full output if no "Final Answer" marker found

In [ ]:
# =============================================================================
# TEST: Travel Query - Kolkata
# =============================================================================
# The agent should:
# 1. Search for local attractions in Kolkata
# 2. Get the current weather in Kolkata
# 3. Combine both into a helpful travel response
# =============================================================================

from IPython.display import display, Markdown

query = "Kolkata"
print(f"🗺️ Query: {query}")
print("="*60)

# Invoke the agent
resp = agent_executor.invoke({"query": query})

# Display the full agent response (includes reasoning chain)
print("\n📋 Full Agent Response:")
display(Markdown(resp["output"]))

In [ ]:
# =============================================================================
# DISPLAY: Final Answer Only
# =============================================================================
# For a cleaner presentation, show only the final answer

final_answer = extract_final_answer(resp)

print("\n🎯 Final Answer:")
display(Markdown(final_answer))

---
## 📝 Summary and Assignment Completion

### What You Built
An **Intelligent Travel Assistant** that:
- ✅ Uses web search to find local attractions
- ✅ Fetches real-time weather data
- ✅ Combines both into helpful travel information
- ✅ Follows the ReAct reasoning pattern

### Key Concepts Practiced
| Concept | Implementation |
|---------|---------------|
| Custom Tools | `@tool` decorator with Tavily and WeatherAPI |
| System Prompt | ReAct pattern + travel-specific instructions |
| Tool Calling Agent | `create_tool_calling_agent` with GPT-4o-mini |
| AgentExecutor | Runtime with early stopping and max iterations |

### Bonus Exercises
1. **Add Memory**: Use `SQLChatMessageHistory` to remember user preferences
2. **More Tools**: Add tools for flight prices, hotel recommendations, or restaurant search
3. **Multi-City**: Extend the agent to compare multiple destinations
4. **Streaming**: Use `stream` instead of `invoke` for real-time responses

In [ ]:
# =============================================================================
# BONUS: Try Other Destinations!
# =============================================================================
# Uncomment and modify to test with different cities

# destinations = ["Mumbai", "Delhi", "Jaipur", "Goa", "Varanasi"]
# 
# for destination in destinations:
#     print(f"\n🗺️ Query: {destination}")
#     print("="*60)
#     resp = agent_executor.invoke({"query": destination})
#     final_answer = extract_final_answer(resp)
#     display(Markdown(final_answer))
#     print("\n")

# LLM Reasoning Implementation Analysis

Based on the provided code, here's a detailed explanation of how the LLM is used for reasoning:

## Overview

This implementation creates an **agent-based reasoning system** using LangChain that follows a **ReAct (Reasoning + Acting) pattern**. The LLM (GPT-4o-mini) acts as a reasoning engine that can think through problems step-by-step and take actions using available tools.

## Core Reasoning Architecture

### 1. **ReAct Framework Implementation**

The system implements a classic ReAct loop where the LLM:

- **Thinks** about the problem
- **Acts** by calling appropriate tools
- **Observes** the results
- **Repeats** until reaching a conclusion

### 2. **Structured Reasoning Prompt**

The reasoning is guided by a detailed system prompt that enforces a specific workflow:

```
Question → Thought → Action → Action Input → Observation → [Loop] → Final Answer
```

### 3. **How the Reasoning Step Works**

#### **Step 1: Problem Analysis**

- The LLM receives a user query and analyzes what information is needed
- It determines whether it can answer with existing knowledge or needs external data

#### **Step 2: Strategic Planning**

- The LLM breaks down complex queries into smaller, manageable steps
- For example, if asked about "Local Attractions in Bangalore and Weather", it identifies two separate tasks

#### **Step 3: Tool Selection**

The LLM has access to two specialized tools:

- **`search_web_extract_info`**: For gathering information from web sources
- **`get_weather`**: For retrieving current weather data

#### **Step 4: Iterative Execution**

- The LLM calls appropriate tools based on its reasoning
- It processes the results and determines if additional actions are needed
- Maximum of 10 iterations prevents infinite loops

#### **Step 5: Synthesis and Response**

- After gathering all necessary information, the LLM synthesizes the data
- It provides a comprehensive "Final Answer" that addresses the original query

## Key Reasoning Features

### **Multi-Step Reasoning**

- The agent can handle complex queries requiring multiple information sources
- It maintains context across multiple tool calls and reasoning steps

### **Adaptive Strategy**

- The LLM can decide between using its trained knowledge vs. calling external tools
- It adapts its approach based on the type of information needed

### **Error Handling and Recovery**

- The system includes timeout mechanisms and error handling for tool calls
- The reasoning process can continue even if some tool calls fail

### **Structured Output**

- The reasoning process is transparent, showing each thought and action
- The final answer is clearly separated from the reasoning process

## Example Reasoning Flow

For a query like "Kolkata":

1. **Thought**: "The user mentioned Kolkata. I should provide local attractions and current weather"
2. **Action**: Call `search_web_extract_info` for Kolkata attractions
3. **Observation**: Process web search results
4. **Action**: Call `get_weather` for Kolkata weather
5. **Observation**: Process weather data
6. **Final Answer**: Synthesize both pieces of information into a comprehensive response

## Technical Implementation Details

- **LLM Model**: GPT-4o-mini with temperature=0 for consistent reasoning
- **Framework**: LangChain's AgentExecutor with tool-calling capabilities
- **Concurrency**: Parallel processing of web content extraction using ThreadPoolExecutor
- **Safety**: Early stopping mechanism and iteration limits prevent runaway processes

This implementation demonstrates sophisticated **multi-modal reasoning** where the LLM acts as an orchestrator, combining its inherent knowledge with real-time data retrieval capabilities to provide comprehensive, well-reasoned responses.
